In [ ]:
from pyspark.sql import (
    SparkSession,
    types,
    functions as F,
)

spark = (
    SparkSession
    .builder
    .appName('cleaning_orders_dataset_with_pyspark')
    .getOrCreate()
)

24/10/30 17:04:40 WARN Utils: Your hostname, codespaces-56c3ba resolves to a loopback address: 127.0.0.1; using 10.0.1.247 instead (on interface eth0)
24/10/30 17:04:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/30 17:04:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/10/30 17:04:54 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
orders_data = spark.read.parquet('orders_data.parquet')
orders_data.toPandas().head()

,order_date,order_id,product,product_id,category,purchase_address,quantity_ordered,price_each,cost_price,turnover,margin
0,2023-01-22 21:25:00,141234,iPhone,5.638009e+12,Vêtements,"944 Walnut St, Boston, MA 02215",1,700.00,231.0000,700.00,469.0000
1,2023-01-28 14:15:00,141235,Lightning Charging Cable,5.563320e+12,Alimentation,"185 Maple St, Portland, OR 97035",1,14.95,7.4750,14.95,7.4750
2,2023-01-17 13:33:00,141236,Wired Headphones,2.113973e+12,Vêtements,"538 Adams St, San Francisco, CA 94016",2,11.99,5.9950,23.98,11.9900
3,2023-01-05 20:33:00,141237,27in FHD Monitor,3.069157e+12,Sports,"738 10th St, Los Angeles, CA 90001",1,149.99,97.4935,149.99,52.4965
4,2023-01-25 11:59:00,141238,Wired Headphones,9.692681e+12,Électronique,"387 10th St, Austin, TX 73301",1,11.99,5.9950,11.99,5.9950


In [6]:
orders_data.createOrReplaceTempView('orders_data')

final_df = spark.sql("""select date(order_date) as order_date
                          ,case when hour(order_date) >= 18 then 'evening'
                            when hour(order_date) >= 12 then 'afternoon'
                            when hour(order_date) >= 5 then 'morning'
                          end time_of_day
                          ,order_id
                          ,lower(product) product
                          ,product_id
                          ,lower(category) category
                          ,purchase_address
                          ,split(split(purchase_address, ', ')[2], ' ')[0] purchase_state
                          ,quantity_ordered
                          ,price_each
                          ,cost_price
                          ,turnover
                          ,margin
                        from orders_data
                        where hour(order_date) >= 5
                          and lower(product) not like '%tv%' """)

final_df.toPandas().to_parquet('orders_data_clean.parquet')